<a href="https://www.kaggle.com/code/aisuko/evaluating-llms-on-any-dataset-using-semscore?scriptVersionId=167701139" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Overview

We evaluated LLMs outputs by using SemScore in [Arena Leaderbaord with SemScore](https://www.kaggle.com/code/aisuko/arena-leaderboard-with-semscore) which is computed sentences similarity of the output of the LLMs. In this notebook, we we going to evaluate models and dataset using SemScore after and during training

In [1]:
%%capture
!pip install transformers==4.38.2
!pip install datasets==2.18.0

In [2]:
!python -m pip freeze

absl-py==1.4.0
accelerate==0.27.2
access==1.1.9
affine==2.4.0
aiobotocore==2.11.2
aiofiles==22.1.0
aiohttp @ file:///home/conda/feedstock_root/build_artifacts/aiohttp_1701099469104/work
aiohttp-cors==0.7.0
aioitertools==0.11.0
aiorwlock==1.3.0
aiosignal @ file:///home/conda/feedstock_root/build_artifacts/aiosignal_1667935791922/work
aiosqlite==0.19.0
albumentations==1.4.0
alembic==1.13.1
altair==5.2.0
annotated-types @ file:///home/conda/feedstock_root/build_artifacts/annotated-types_1696634205638/work
annoy==1.17.3
anyio @ file:///home/conda/feedstock_root/build_artifacts/anyio_1702909220329/work
apache-beam==2.46.0
aplus==0.11.0
appdirs==1.4.4
archspec @ file:///home/conda/feedstock_root/build_artifacts/archspec_1699370045702/work
argon2-cffi @ file:///home/conda/feedstock_root/build_artifacts/argon2-cffi_1692818318753/work
argon2-cffi-bindings @ file:///home/conda/feedstock_root/build_artifacts/argon2-cffi-bindings_1695386546427/work
array-record==0.5.0
arrow @ file:///home/conda/fe

In [3]:
!transformers-cli env

2024-03-18 23:06:27.409369: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-18 23:06:27.409484: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-18 23:06:27.551512: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered

Copy-and-paste the text below in your GitHub issue and FILL OUT the two last points.

- `transformers` version: 4.38.2
- Platform: Linux-5.15.133+-x86_64-with-glibc2.31
- Python version: 3.10.13
- Huggingface_hub version: 0.20.3
- Safetensors version: 0.4.2
- Accelerate version: 0.27.2
- Accelerate config: 	not found
- PyTorch version (GPU?): 2.1.2 (True)
- Tens

In [4]:
import os
import torch
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
login(token=user_secrets.get_secret("HUGGINGFACE_TOKEN"))

os.environ["MODEL"]="sentence-transformers/all-mpnet-base-v2"
os.environ["DATASET"]="g-ronimo/oasst2_top1_en"

torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Loading Model and Calculating Semantic Similarity

Here we will use `sentence-transformers/all-mpnet-base-v2`.

In [5]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel

# mean pooling take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings=model_output[0] # first element to model_output contains all token embeddings
    input_mask_expanded=attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings*input_mask_expanded,1)/torch.clamp(input_mask_expanded.sum(1), min=1e-9)

tokenizer=AutoTokenizer.from_pretrained(os.getenv("MODEL"))

encoder=AutoModel.from_pretrained(os.getenv("MODEL"))
encoder.max_seq_length=5000
encoder.to("cuda")

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


MPNetModel(
  (embeddings): MPNetEmbeddings(
    (word_embeddings): Embedding(30527, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): MPNetEncoder(
    (layer): ModuleList(
      (0-11): 12 x MPNetLayer(
        (attention): MPNetAttention(
          (attn): MPNetSelfAttention(
            (q): Linear(in_features=768, out_features=768, bias=True)
            (k): Linear(in_features=768, out_features=768, bias=True)
            (v): Linear(in_features=768, out_features=768, bias=True)
            (o): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (intermediate): MPNetIntermediate(
          (dense): Linear(in_

In [6]:
torch.manual_seed(2024)

sentences=["apple","orange","car"]

encoded_input=tokenizer(sentences, padding=True, truncation=True, return_tensors="pt").to('cuda')

# compute token embeddings
with torch.no_grad():
    model_output=encoder(**encoded_input)

sentence_embeddings=mean_pooling(model_output, encoded_input['attention_mask'])

# normalize embeddings
sentence_embeddings=F.normalize(sentence_embeddings, p=2, dim=1)

# To obtain the cosine similarity, we calculate the dot product of the normalized vectors
for i in range(0, len(sentences)):
    print(sentences[0], sentences[i], (sentence_embeddings[0] @ sentence_embeddings[i]).item())

apple apple 0.9999998807907104
apple orange 0.40115001797676086
apple car 0.31371933221817017


# Evaluating Model on Any Dataset

In [7]:
import torch.nn as nn
from tqdm import tqdm

def emb_mean_pooling(embedding, attention_mask):
    token_embeddings=embedding[0]
    input_mask_expanded=attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings*input_mask_expanded,1)/torch.clamp(input_mask_expanded.sum(1),min=1e-9)


def get_embeddings(sentences, b_size=8):
    embeddings=torch.tensor([], device='cuda')
    batches=[sentences[i:i+b_size] for i in range(0, len(sentences), b_size)]
    
    for s in batches:
        encoded_input=tokenizer(s, padding=True, truncation=True, return_tensors='pt').to('cuda')
        with torch.no_grad():
            output=encoder(**encoded_input)
        batch_embeddings=emb_mean_pooling(output, encoded_input['attention_mask'])
        embeddings=torch.cat((embeddings, batch_embeddings), dim=0)
    return embeddings


def nn_cos(emd1,emd2=None):
    cos=nn.CosineSimilarity(dim=1, eps=1e-6)
    if emd2 is None:
        num_samples=emd1.shape[0]
        similarities=[[0 for i in range(num_samples)] for f in range(num_samples)]
        for row in tqdm(range(num_samples)):
            similarities[row][0:row+1]=cos(emd1[row].repeat(row+1,1), emd1[0:row+1]).tolist()
        return similarities
    else:
        return cos(emd_ans, emd_ref).tolist()


words=['lemon', 'orange', 'car','money']
embds=get_embeddings(words)
similarity=nn_cos(embds)
print(similarity)

100%|██████████| 4/4 [00:00<00:00, 1073.88it/s]

[[1.0, 0, 0, 0], [0.5340331792831421, 1.0, 0, 0], [0.2909420132637024, 0.30266571044921875, 1.0, 0], [0.22813823819160461, 0.19676585495471954, 0.3411044478416443, 1.0]]


In [8]:
from datasets import load_dataset

dataset=load_dataset(os.getenv("DATASET"), split="train")
dataset=dataset.shuffle(seed=42).select(range(10))
dataset

Generating train split:   0%|          | 0/5419 [00:00<?, ? examples/s]

Dataset({
    features: ['conversation'],
    num_rows: 10
})

In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer

os.environ["TINYLLAMA"]="TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer=AutoTokenizer.from_pretrained(os.getenv("TINYLLAMA"), use_fast=False)
model=AutoModelForCausalLM.from_pretrained(os.getenv("TINYLLAMA"), device_map="auto", torch_dtype=torch.bfloat16, trust_remote_code=True)
model.device

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

device(type='cuda', index=0)

We use TinyLlama to generate answers, and create a list to save the reference answers. We will calcualte the semantic similarity of these two answers.

In [11]:
answers_ref, answers_pred = [], []

gen_config={
    "temperature":0.7,
    "top_p":0.1,
    "repetition_penalty":1.18,
    "top_k":40,
    "do_sample":True,
    "max_new_tokens": 500
}

for conversation in dataset["conversation"]:
    for i, msg in enumerate(conversation):
        if msg["role"] == "assistant": continue
        partial_conv = conversation[0:i+1]
        prompt_tok = tokenizer.apply_chat_template(
            partial_conv, 
            add_generation_prompt = True, 
            return_tensors = "pt"
        ).to("cuda")
        answer_tok = model.generate(
            prompt_tok,
            **gen_config,
            eos_token_id = tokenizer.eos_token_id 
        )
        answer = tokenizer.decode(
            answer_tok[0][len(prompt_tok[0]):], 
            skip_special_tokens = True
        )
        answers_pred.append(answer) 
        answers_ref.append(conversation[i+1]["content"])

In [14]:
from statistics import mean

emd_ans=get_embeddings(answers_pred)
emd_ref=get_embeddings(answers_ref)

similarities=nn_cos(emd_ans, emd_ref)

for i, result in enumerate(similarities):
    print(f"question {i}:{similarities[i]}")

print("avg, similarity pred. vs. ref.:", mean(similarities), "\n")

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


# Acknowledge

* https://medium.com/@geronimo7/semscore-evaluating-llms-with-semantic-similarity-2abf5c2fadb9
* https://github.com/geronimi73/semscore/blob/main/nb_blog_part2_eval-trained.ipynb